In [1]:
import numpy as np
import pandas as pd
import os
import random
import time
import csv
# ML imports
from sklearn.model_selection import KFold, train_test_split, cross_val_score, cross_val_predict, GridSearchCV 
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.decomposition import PCA, IncrementalPCA
import statsmodels.api as sm
# Visualizations
import matplotlib.pyplot as plt  # static plotting
import seaborn as sns  # pretty plotting, including heat map
%matplotlib inline

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
process_time = []

In [3]:
os.getcwd()

'C:\\QGIS data\\web-mapping-application'

In [4]:
# df_train4=pd.read_csv(r"data\NewTable08_Map_Crime_MY1020110801imblearn-year1-5.csv")
df_train4=pd.read_csv(r"data\NewTable08_Map_Crime_MY1020110801imblearn-6.csv")
# df_test4_M14=pd.read_csv(r"data\NewTable08_Map_Crime_MY1020110801imblearn-year6.csv")

In [5]:
df_train4.head()

,num_row,Month,Month_La,Year,Area_unitID,Area_Unit_TA,Day Of Week ID,Day_Of_Week,HRGroup,Time,TA_ID,Locn_Type_DivisionID,Locn_Type_Division,Types of Crime ID,Types_of_Crime,Crime_cases
0,1429005,11,November,2,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.0,1,Community Location,6,Theft and Related Offences,6
1,1429006,2,February,3,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.0,1,Community Location,6,Theft and Related Offences,6
2,1429007,11,November,2,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.0,1,Community Location,6,Theft and Related Offences,6
3,1429008,1,January,3,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.0,1,Community Location,6,Theft and Related Offences,6
4,1429009,2,February,3,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.0,1,Community Location,6,Theft and Related Offences,6


In [6]:
df_train4_loaded=df_train4.drop(['num_row','Month','Year','Area_unitID','Day Of Week ID','HRGroup','TA_ID','Locn_Type_DivisionID','Types of Crime ID'],axis=1)
df_train4_loaded_drop=df_train4_loaded.dropna()
df_train4_loaded_drop

,Month_La,Area_Unit_TA,Day_Of_Week,Time,Locn_Type_Division,Types_of_Crime,Crime_cases
0,November,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
1,February,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
2,November,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
3,January,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
4,February,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
...,...,...,...,...,...,...,...
5453226,June,"Pinehill, Auckland",UNKNOWN,00:00-23:59,Other Location,"Unlawful Entry With Intent/Burglary, Break and...",9
5453227,June,"Pinehill, Auckland",UNKNOWN,00:00-23:59,Other Location,"Unlawful Entry With Intent/Burglary, Break and...",9
5453228,March,"Pinehill, Auckland",UNKNOWN,00:00-23:59,Not Applicable,"Unlawful Entry With Intent/Burglary, Break and...",9
5453229,June,"Pinehill, Auckland",UNKNOWN,00:00-23:59,Other Location,"Unlawful Entry With Intent/Burglary, Break and...",9


In [7]:
for col in df_train4_loaded_drop.columns:
    print(col,len(df_train4[col].unique()),df_train4[col].unique())

Month_La 12 ['November' 'February' 'January' 'October' 'May' 'June' 'April'
 'September' 'August' 'July' 'March' 'December']
Area_Unit_TA 1932 ['Mt Eden South, Auckland' 'Mosgiel South, Dunedin City'
 'Mt Hobson, Auckland' ... 'Cambridge North, Waipa District'
 'Cape Rodney, Auckland' 'Cape Rodney South, Auckland']
Day_Of_Week 9 ['UNKNOWN' 'Friday' 'Monday' 'Saturday' 'Sunday' 'Tuesday' 'Thursday'
 'Wednesday' nan]
Time 8 ['00:00-23:59' '16:00-19:59' '20:00-23:59' '08.00-11:59' '12:00-15:59' nan
 '04:00-07:59' '00:00-03.59']
Locn_Type_Division 5 ['Community Location' 'Residential Location' 'Other Location'
 'Not Applicable' 'Unspecified Location']
Types_of_Crime 6 ['Theft and Related Offences'
 'Unlawful Entry With Intent/Burglary, Break and Enter'
 'Acts Intended to Cause Injury' 'Sexual Assault and Related Offences'
 'Robbery, Extortion and Related Offences'
 'Abduction, Harassment and Other Related Offences']
Crime_cases 10 [ 6  7  4  5  1  2  8  9  3 10]


In [8]:
# from sklearn.preprocessing import LabelEncoder
# le_Area_Unit =LabelEncoder()
# df_train4['Area_Unit']=le_Area_Unit.fit_transform(df_train4['Area_Unit'])
# df_train4["Area_Unit"].unique()

d= {v: k for k, v in enumerate(set(df_train4['Area_Unit_TA'].unique()))} 
d.items()

dict_items([('Newlands South, Wellington City', 0), ('Ohakea, Manawatu District', 1), ('Clendon South, Auckland', 2), ('Sandringham North, Auckland', 3), ('Inlet-Tasman Bay, Nelson City', 4), ('Kingdale, Auckland', 5), ('Mahia, Wairoa District', 6), ('Waverley, South Taranaki District', 7), ('Laird Park, Wanganui District', 8), ('Leigh, Auckland', 9), ('Pakuranga North, Auckland', 10), ("Otane, Central Hawke's Bay District", 11), ('Marlborough Sounds Coastal Marine, Marlborough District', 12), ('Enderley, Hamilton City', 13), ('Opaki-Fernridge, Masterton District', 14), ('Shelly Park, Auckland', 15), ('Mclean Park, Napier City', 16), ('Somerfield, Christchurch City', 17), ('Saxton, Nelson City', 18), ('Maori Hill, Dunedin City', 19), ('Otara North, Auckland', 20), ('Sandringham East, Auckland', 21), ('Kaiapoi West, Waimakariri District', 22), ('Clendon North, Auckland', 23), ('Inangahua Valley, Buller District', 24), ('Netherby, Ashburton District', 25), ('Favona North, Auckland', 26),

In [9]:
def generate_dict(data):
    my_dict= {v:k for k, v in enumerate(set(data))}
    return my_dict

In [10]:
obj_list=['Area_Unit_TA']
obj_names=['d_Area_Unit']
for i,j in zip(obj_names,obj_list):
    print('{}=generate_dict["{}"]'.format(i,j))

d_Area_Unit=generate_dict["Area_Unit_TA"]


In [11]:
df_train4_loaded_drop.columns

Index(['Month_La', 'Area_Unit_TA', 'Day_Of_Week', 'Time', 'Locn_Type_Division',
       'Types_of_Crime', 'Crime_cases'],
      dtype='object')

In [12]:
d_Month_La=generate_dict(df_train4_loaded_drop["Month_La"].unique())
d_Area_Unit=generate_dict(df_train4_loaded_drop["Area_Unit_TA"].unique())
d_Day_Of_Week=generate_dict(df_train4_loaded_drop["Day_Of_Week"].unique())
d_Time=generate_dict(df_train4_loaded_drop["Time"].unique())
d_Locn_Type_Division=generate_dict(df_train4_loaded_drop["Locn_Type_Division"].unique())
d_Types_of_Crime=generate_dict(df_train4_loaded_drop["Types_of_Crime"].unique())

In [13]:
d_Month_La

{'July': 0,
 'May': 1,
 'December': 2,
 'January': 3,
 'September': 4,
 'October': 5,
 'February': 6,
 'November': 7,
 'March': 8,
 'April': 9,
 'August': 10,
 'June': 11}

In [14]:
df_train4_loaded_drop

,Month_La,Area_Unit_TA,Day_Of_Week,Time,Locn_Type_Division,Types_of_Crime,Crime_cases
0,November,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
1,February,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
2,November,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
3,January,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
4,February,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
...,...,...,...,...,...,...,...
5453226,June,"Pinehill, Auckland",UNKNOWN,00:00-23:59,Other Location,"Unlawful Entry With Intent/Burglary, Break and...",9
5453227,June,"Pinehill, Auckland",UNKNOWN,00:00-23:59,Other Location,"Unlawful Entry With Intent/Burglary, Break and...",9
5453228,March,"Pinehill, Auckland",UNKNOWN,00:00-23:59,Not Applicable,"Unlawful Entry With Intent/Burglary, Break and...",9
5453229,June,"Pinehill, Auckland",UNKNOWN,00:00-23:59,Other Location,"Unlawful Entry With Intent/Burglary, Break and...",9


In [15]:
df2=df_train4_loaded_drop

In [16]:
df2.head(5)

,Month_La,Area_Unit_TA,Day_Of_Week,Time,Locn_Type_Division,Types_of_Crime,Crime_cases
0,November,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
1,February,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
2,November,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
3,January,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
4,February,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6


In [17]:
df2["Area_Unit_TA"]=df2["Area_Unit_TA"].map(d_Area_Unit)
df2["Month_La"]=df2["Month_La"].map(d_Month_La)
df2["Day_Of_Week"]=df2["Day_Of_Week"].map(d_Day_Of_Week)
df2["Time"]=df2["Time"].map(d_Time)
df2["Locn_Type_Division"]=df2["Locn_Type_Division"].map(d_Locn_Type_Division)
df2["Types_of_Crime"]=df2["Types_of_Crime"].map(d_Types_of_Crime)

C:\Users\admin\AppData\Local\Temp\ipykernel_1980\3222048269.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Area_Unit_TA"]=df2["Area_Unit_TA"].map(d_Area_Unit)
C:\Users\admin\AppData\Local\Temp\ipykernel_1980\3222048269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Month_La"]=df2["Month_La"].map(d_Month_La)
C:\Users\admin\AppData\Local\Temp\ipykernel_1980\3222048269.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [18]:
df2.head(5)
print (df2.isnull().sum())

Month_La              0
Area_Unit_TA          0
Day_Of_Week           0
Time                  0
Locn_Type_Division    0
Types_of_Crime        0
Crime_cases           0
dtype: int64


In [19]:
df2.dtypes
# df2_drop=df2.dropna()

Month_La              int64
Area_Unit_TA          int64
Day_Of_Week           int64
Time                  int64
Locn_Type_Division    int64
Types_of_Crime        int64
Crime_cases           int64
dtype: object

In [20]:
df_train4.head()

,num_row,Month,Month_La,Year,Area_unitID,Area_Unit_TA,Day Of Week ID,Day_Of_Week,HRGroup,Time,TA_ID,Locn_Type_DivisionID,Locn_Type_Division,Types of Crime ID,Types_of_Crime,Crime_cases
0,1429005,11,November,2,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.0,1,Community Location,6,Theft and Related Offences,6
1,1429006,2,February,3,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.0,1,Community Location,6,Theft and Related Offences,6
2,1429007,11,November,2,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.0,1,Community Location,6,Theft and Related Offences,6
3,1429008,1,January,3,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.0,1,Community Location,6,Theft and Related Offences,6
4,1429009,2,February,3,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.0,1,Community Location,6,Theft and Related Offences,6


In [21]:
df_train4.dtypes

num_row                   int64
Month                     int64
Month_La                 object
Year                      int64
Area_unitID               int64
Area_Unit_TA             object
Day Of Week ID            int64
Day_Of_Week              object
HRGroup                   int64
Time                     object
TA_ID                   float64
Locn_Type_DivisionID      int64
Locn_Type_Division       object
Types of Crime ID         int64
Types_of_Crime           object
Crime_cases               int64
dtype: object

In [22]:
df_train4.dropna()

,num_row,Month,Month_La,Year,Area_unitID,Area_Unit_TA,Day Of Week ID,Day_Of_Week,HRGroup,Time,TA_ID,Locn_Type_DivisionID,Locn_Type_Division,Types of Crime ID,Types_of_Crime,Crime_cases
0,1429005,11,November,2,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,1,Community Location,6,Theft and Related Offences,6
1,1429006,2,February,3,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,1,Community Location,6,Theft and Related Offences,6
2,1429007,11,November,2,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,1,Community Location,6,Theft and Related Offences,6
3,1429008,1,January,3,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,1,Community Location,6,Theft and Related Offences,6
4,1429009,2,February,3,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,1,Community Location,6,Theft and Related Offences,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5453226,5264205,6,June,3,880,"Pinehill, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,3,Other Location,5,"Unlawful Entry With Intent/Burglary, Break and...",9
5453227,5264206,6,June,3,880,"Pinehill, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,3,Other Location,5,"Unlawful Entry With Intent/Burglary, Break and...",9
5453228,5264207,3,March,4,880,"Pinehill, Auckland",99,UNKNOWN,8,00:00-23:59,40.651015,2,Not Applicable,5,"Unlawful Entry With Intent/Burglary, Break and...",9
5453229,5264208,6,June,3,880,"Pinehill, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,3,Other Location,5,"Unlawful Entry With Intent/Burglary, Break and...",9


In [23]:
df2.isnull().sum()

Month_La              0
Area_Unit_TA          0
Day_Of_Week           0
Time                  0
Locn_Type_Division    0
Types_of_Crime        0
Crime_cases           0
dtype: int64

In [24]:
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5437710 entries, 0 to 5453230
Data columns (total 7 columns):
 #   Column              Dtype
---  ------              -----
 0   Month_La            int64
 1   Area_Unit_TA        int64
 2   Day_Of_Week         int64
 3   Time                int64
 4   Locn_Type_Division  int64
 5   Types_of_Crime      int64
 6   Crime_cases         int64
dtypes: int64(7)
memory usage: 331.9 MB
None


In [25]:
df2["Month_La"] = df2['Month_La'].astype('int32')
df2["Area_Unit_TA"] = df2['Area_Unit_TA'].astype('int32')
df2["Day_Of_Week"] = df2['Day_Of_Week'].astype('int32')
df2["Time"] = df2['Time'].astype('int32')
df2["Locn_Type_Division"] = df2['Locn_Type_Division'].astype('int32')
df2["Types_of_Crime"] = df2['Types_of_Crime'].astype('int32')
df2["Crime_cases"] = df2['Crime_cases'].astype('int32')
df2.dtypes

C:\Users\admin\AppData\Local\Temp\ipykernel_1980\2869146917.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Month_La"] = df2['Month_La'].astype('int32')
C:\Users\admin\AppData\Local\Temp\ipykernel_1980\2869146917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Area_Unit_TA"] = df2['Area_Unit_TA'].astype('int32')
C:\Users\admin\AppData\Local\Temp\ipykernel_1980\2869146917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Month_La              int32
Area_Unit_TA          int32
Day_Of_Week           int32
Time                  int32
Locn_Type_Division    int32
Types_of_Crime        int32
Crime_cases           int32
dtype: object

In [26]:
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5437710 entries, 0 to 5453230
Data columns (total 7 columns):
 #   Column              Dtype
---  ------              -----
 0   Month_La            int32
 1   Area_Unit_TA        int32
 2   Day_Of_Week         int32
 3   Time                int32
 4   Locn_Type_Division  int32
 5   Types_of_Crime      int32
 6   Crime_cases         int32
dtypes: int32(7)
memory usage: 186.7 MB
None


In [27]:
df_train4_loaded=df_train4.drop(['Area_unitID'],axis=1)

In [28]:
df_train4.dropna()

,num_row,Month,Month_La,Year,Area_unitID,Area_Unit_TA,Day Of Week ID,Day_Of_Week,HRGroup,Time,TA_ID,Locn_Type_DivisionID,Locn_Type_Division,Types of Crime ID,Types_of_Crime,Crime_cases
0,1429005,11,November,2,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,1,Community Location,6,Theft and Related Offences,6
1,1429006,2,February,3,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,1,Community Location,6,Theft and Related Offences,6
2,1429007,11,November,2,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,1,Community Location,6,Theft and Related Offences,6
3,1429008,1,January,3,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,1,Community Location,6,Theft and Related Offences,6
4,1429009,2,February,3,1462,"Mt Eden South, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,1,Community Location,6,Theft and Related Offences,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5453226,5264205,6,June,3,880,"Pinehill, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,3,Other Location,5,"Unlawful Entry With Intent/Burglary, Break and...",9
5453227,5264206,6,June,3,880,"Pinehill, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,3,Other Location,5,"Unlawful Entry With Intent/Burglary, Break and...",9
5453228,5264207,3,March,4,880,"Pinehill, Auckland",99,UNKNOWN,8,00:00-23:59,40.651015,2,Not Applicable,5,"Unlawful Entry With Intent/Burglary, Break and...",9
5453229,5264208,6,June,3,880,"Pinehill, Auckland",99,UNKNOWN,8,00:00-23:59,3.000000,3,Other Location,5,"Unlawful Entry With Intent/Burglary, Break and...",9


In [29]:
df2.columns
df2.head()
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5437710 entries, 0 to 5453230
Data columns (total 7 columns):
 #   Column              Dtype
---  ------              -----
 0   Month_La            int32
 1   Area_Unit_TA        int32
 2   Day_Of_Week         int32
 3   Time                int32
 4   Locn_Type_Division  int32
 5   Types_of_Crime      int32
 6   Crime_cases         int32
dtypes: int32(7)
memory usage: 186.7 MB
None


In [30]:
# df2_15 = df2[df2['Month_La'] != 11]
# df2_test = df2[df2['Month_La'] == 11]

In [31]:
train, test = train_test_split(df2, test_size=0.2, random_state=25)
print(f"No. of training examples: {train.shape[0]}")
print(f"No. of testing examples: {test.shape[0]}")

No. of training examples: 4350168
No. of testing examples: 1087542


In [32]:
d_Month_La
df2['Month_La']

0           7
1           6
2           7
3           3
4           6
           ..
5453226    11
5453227    11
5453228     8
5453229    11
5453230    11
Name: Month_La, Length: 5437710, dtype: int32

In [33]:
 # Assign classifier
cls_rf = RandomForestClassifier(random_state=1, n_jobs=-1, n_estimators=90)
# RandomForestRegressor
# Split train and test set 90/10
X = df2[['Month_La', 'Area_Unit_TA', 'Day_Of_Week', 'Time', 'Locn_Type_Division',
       'Types_of_Crime' ]]
y =df2[['Crime_cases']]
X_test = df2[['Month_La', 'Area_Unit_TA', 'Day_Of_Week', 'Time', 'Locn_Type_Division',
       'Types_of_Crime' ]]
y_test  = df2[['Crime_cases']]
# X_new = df2_test[['Month_La', 'Area_Unit_TA', 'Day_Of_Week', 'Time', 'Locn_Type_Division',
#        'Types_of_Crime']]
# y_new =df2_test[['Crime_cases']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=50)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4350168, 6)
(4350168, 1)
(1087542, 6)
(1087542, 1)


In [136]:
X

,Month_La,Area_Unit_TA,Day_Of_Week,Time,Locn_Type_Division,Types_of_Crime
0,6,623,6,6,4,2
1,11,623,6,6,4,2
2,6,623,6,6,4,2
3,9,623,6,6,4,2
4,11,623,6,6,4,2
...,...,...,...,...,...,...
5453226,8,1614,6,6,1,3
5453227,8,1614,6,6,1,3
5453228,7,1614,6,6,2,3
5453229,8,1614,6,6,1,3


In [34]:
# Fit model and run on test set with timing
start_time = time.time()
result= cls_rf.fit(X_train, y_train.values.ravel()),
print('Accuracy: ', cls_rf.score(X_test, y_test)),

# Timing
elapsed_time = time.time() - start_time
print('Formatted time: ', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
print('Time in seconds: ', elapsed_time)
process_time.append(elapsed_time)

Accuracy:  0.8146287683602105
Formatted time:  00:04:14
Time in seconds:  254.66162967681885


In [35]:
pred = cls_rf.predict(X_test)
print('F1 accuracy: ', f1_score(pred, y_test, average='macro'))
    # rfc_pred = rfc.predict(X_test)
print (classification_report(y_test,pred))

F1 accuracy:  0.8140906591456488
              precision    recall  f1-score   support

           1       0.95      0.84      0.89    108697
           2       0.83      0.75      0.79    108119
           3       0.81      0.72      0.77    108461
           4       0.78      0.74      0.76    108683
           5       0.78      0.78      0.78    109357
           6       0.78      0.80      0.79    108981
           7       0.80      0.85      0.82    108547
           8       0.81      0.86      0.83    108864
           9       0.83      0.91      0.87    108859
          10       0.79      0.89      0.84    108974

    accuracy                           0.81   1087542
   macro avg       0.82      0.81      0.81   1087542
weighted avg       0.82      0.81      0.81   1087542



In [ ]:
# # X_test_
# print(X_new.shape)
# result1=cls_rf.predict(X_new)
# # score=best_fit.score(X_test,y_test)*100
# score1=cls_rf.score(X_new, y_new)

In [36]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import RobustScaler

In [37]:
models = pd.DataFrame(index=['train_mse','test_mse','train_rmse','test_rmse'],
                     columns = ['cls_rf'])

In [38]:
models.loc['train_mse','cls_rf']= mean_squared_error(y_pred=cls_rf.predict(X_train),y_true=y_train)
models.loc['test_mse','cls_rf']= mean_squared_error(y_pred=cls_rf.predict(X_test),y_true=y_test)
models.loc['train_rmse','cls_rf']= np.sqrt(mean_squared_error(y_pred=cls_rf.predict(X_train),y_true=y_train))
models.loc['test_rmse','cls_rf']= np.sqrt(mean_squared_error(y_pred=cls_rf.predict(X_test),y_true=y_test))

In [39]:
models

,cls_rf
train_mse,1.292003
test_mse,1.329166
train_rmse,1.136663
test_rmse,1.152895


In [41]:
from sklearn.feature_selection import SelectKBest, chi2

X = df2[['Month_La', 'Area_Unit_TA', 'Day_Of_Week', 'Time', 'Locn_Type_Division',
       'Types_of_Crime' ]]
y = df2[['Crime_cases']]

feature_selector = SelectKBest(chi2, k = "all")
fit = feature_selector.fit(X,y)

p_values = pd.DataFrame(fit.pvalues_)
scores = pd.DataFrame(fit.scores_)
input_variable_names = pd.DataFrame(X.columns)
summary_stats = pd.concat([input_variable_names, p_values, scores], axis = 1)
summary_stats.columns = ["input_variable", "p_value", "chi2_score"]
summary_stats.sort_values(by = "p_value", inplace = True)

p_value_threshold = 0.05
p_value_threshold1 = 0.01
p_value_threshold2 = 0.1
score_threshold = 5

selected_variables = summary_stats.loc[(summary_stats["chi2_score"] >= score_threshold) &
                                       (summary_stats["p_value"] <= p_value_threshold2)]
selected_variables = selected_variables["input_variable"].tolist()
X_new = X[selected_variables]
selected_variables

['Month_La',
 'Area_Unit_TA',
 'Day_Of_Week',
 'Time',
 'Locn_Type_Division',
 'Types_of_Crime']

In [42]:
summary_stats

,input_variable,p_value,chi2_score
0,Month_La,0.0,8.293920e+03
1,Area_Unit_TA,0.0,6.304588e+06
2,Day_Of_Week,0.0,2.398514e+05
3,Time,0.0,2.557780e+05
4,Locn_Type_Division,0.0,1.526099e+05
5,Types_of_Crime,0.0,2.604772e+04


In [43]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy.stats import norm, chi2
from scipy.stats import t as t_dist
from sklearn.datasets import load_wine
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
# Libs implementations
from mlxtend.evaluate import mcnemar
from mlxtend.evaluate import mcnemar_table
from mlxtend.evaluate import paired_ttest_5x2cv
from mlxtend.evaluate import proportion_difference
from mlxtend.evaluate import paired_ttest_kfold_cv
from mlxtend.evaluate import paired_ttest_resampled

In [44]:
X = df2[['Month_La', 'Area_Unit_TA', 'Day_Of_Week', 'Time', 'Locn_Type_Division',
       'Types_of_Crime' ]]
y = df2[['Crime_cases']]

In [45]:
rf = RandomForestClassifier(random_state=42)
knn = KNeighborsClassifier(n_neighbors=1)

In [ ]:
# X[:,1]=le_Area_Unit.transform(X[:,1])
# X=X.astype(float)
# X

In [53]:
# pred = cls_rf.predict(X)
# pred

array([4, 6, 2, ..., 9, 9, 9])

In [ ]:
# X_train_new = cls_rf(X_train,y_train)

In [54]:
#save model
import pickle

# d_Month_La=generate_dict(df_train4_loaded_drop["Month_La"].unique())
# d_Area_Unit=generate_dict(df_train4_loaded_drop["Area_Unit_TA"].unique())
# d_Day_Of_Week=generate_dict(df_train4_loaded_drop["Day_Of_Week"].unique())
# d_Time=generate_dict(df_train4_loaded_drop["Time"].unique())
# d_Locn_Type_Division=generate_dict(df_train4_loaded_drop["Locn_Type_Division"].unique())
# d_Types_of_Crime=generate_dict(df_train4_loaded_drop["Types_of_Crime"].unique())

In [55]:
data= {"model":cls_rf, 
       "d_Area_Unit":d_Area_Unit, 
       "d_Month_La":d_Month_La,"d_Day_Of_Week":d_Day_Of_Week,
       "d_Time":d_Time,
       "d_Locn_Type_Division":d_Locn_Type_Division,
       "d_Types_of_Crime":d_Types_of_Crime
      }

with open('save_steps.pkl','wb') as file:
    pickle.dump(data,file)

In [58]:
with open('save_steps.pkl','rb') as file:
        data= pickle.load(file)

cls_rf_loaded= data["model"]
d_Area_Unit=data["d_Area_Unit"]
d_Month_La=data["d_Month_La"]
d_Day_Of_Week=data["d_Day_Of_Week"]
d_Time=data["d_Time"]
d_Locn_Type_Division=data["d_Locn_Type_Division"]
d_Types_of_Crime=data["d_Types_of_Crime"]

In [59]:
# pred=cls_rf_loaded.predict(X)
# pred

array([4, 6, 2, ..., 9, 9, 9])

In [60]:
#export by joblib
# from sklearn.externals 
import joblib

In [61]:
Crime_case_model= open("Crime_case_model","wb")
joblib.dump(cls_rf,Crime_case_model)

In [62]:
Crime_case_model.close()

In [46]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, f1_score
import sklearn.metrics as metrics

In [47]:
pred_prob = cls_rf.predict_proba(X_test)
roc_auc=roc_auc_score(y_test, pred_prob, multi_class='ovr')

In [48]:
roc_auc

0.9824228305860642

In [49]:
print('Random Forest roc AUC: ', roc_auc)

Random Forest roc AUC:  0.9824228305860642
